<a href="https://colab.research.google.com/github/sallyy1/Data-Analysis/blob/master/NLU(%EA%B0%90%EC%A0%95%20%EB%B6%84%EB%A5%98)_%E1%84%87%E1%85%A1%E1%86%AF%E1%84%92%E1%85%AA_%E1%84%8C%E1%85%A5%E1%86%AB%E1%84%8E%E1%85%A6_%E1%84%80%E1%85%B5%E1%84%8C%E1%85%AE%E1%86%AB_%E1%84%8E%E1%85%AE%E1%84%8E%E1%85%AE%E1%86%AF_tcn%2C_%E1%84%83%E1%85%A2%E1%84%87%E1%85%AE%E1%86%AB%E1%84%85%E1%85%B2_%E1%84%8C%E1%85%A5%E1%86%A8%E1%84%8B%E1%85%AD%E1%86%BC_%E1%84%86%E1%85%A9%E1%84%83%E1%85%AE_%E1%84%8B%E1%85%AA%E1%86%AB%E1%84%85%E1%85%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 임포트

In [1]:
import json # json 파일 불러오기 관련
import tensorflow as tf
import numpy as np
import urllib

from tensorflow.keras.preprocessing.text import Tokenizer # 전처리 관련
from tensorflow.keras.preprocessing.sequence import pad_sequences # 전처리 관련
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten, Conv1D, MaxPooling1D, Dropout, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

from keras import optimizers, losses
from keras.optimizers import Adam

- 데이콘 1위 추가 임포트

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [78]:
#!pip install -U pip
#!pip install pandas
#!pip install -U scikit-learn
#!pip install -U tensorflow
#!pip install -U tensorflow_hub
!pip install -U sentencepiece

     |████████████████████████████████| 1.1MB 12.3MB/s 


In [ ]:
#import tokenization

In [119]:
import gc
from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import sys
sys.path.append('../src')
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.backend import clear_session
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub
#import tokenization
import warnings 
warnings.filterwarnings(action='ignore')

In [5]:
tf.__version__

'2.4.0'

# 경로 설정

In [12]:
train_path = 'train_data.json'
valid_path = 'dev_data.json'

# 데이터 로드

- train 데이터 불러오기

In [13]:
# json 딕셔너리 파일에서 content 데이터만 추출

import json # import json module

train_data = []
train_data_content = []

train_y = []


# train data - 대화내용 'content' & emotion 'type'불러오기
with open(train_path) as json_file:
    json_data = json.load(json_file)

    for i in json_data: # i : dict
        temp = (i.get('talk')).get('content')
        
        if '' not in temp:
            train_data.append(temp) # "content" 만
            
            train_data_content.append(list(temp.values()))
            
            #for v1 in temp.values():
            #    train_data_content += v1
            
            train_y.append(((i.get('profile')).get('emotion')).get('type'))
        
        
    print(train_data[:5], type(train_data[0]))
    print()
    print(train_y[:5], type(train_y[0]))
    print()
    print(train_data_content[:5], type(train_data_content[0]))


[{'HS01': '기분이 좋지 않아', 'SS01': '기분이 좋지 않은 이유를 알 수 있을까요?', 'HS02': '요새 건강검진을 받았는데 건강 상태가 좋지 않아.', 'SS02': '많이 속상하시겠어요. 어디가 안 좋은가요?', 'HS03': '이곳 저곳 관리해야 할 곳이 많아.', 'SS03': '안좋은 곳이 많아 걱정이 많으시겠어요.'}, {'HS01': '진로가 너무 걱정스러워. 내가 진짜 뭘 해야 할지 모르겠어.', 'SS01': '미래가 두려우시군요. 하고 싶은 일이 있으세요?', 'HS02': '하고 싶은 일은 있는데 지금 시작하기에 너무 늦은 것 같아서 선뜻 도전을 못 하겠어.', 'SS02': '도전하는 것에 두려움을 느끼고 계시네요.', 'HS03': '', 'SS03': ''}, {'HS01': '아이가 친구랑 다퉈서 걱정이야.', 'SS01': '걱정이 되시겠군요. 아이는 어떤 상태인가요?', 'HS02': '아이도 걱정이 되는지 밥도 잘 안먹더라고.', 'SS02': '많이 걱정되시겠어요. 어떻게 하면 아이를 도와줄 수 있을까요?', 'HS03': '저녁먹고 대화를 해봐야지.', 'SS03': '대화가 잘 되었으면 좋겠어요.'}, {'HS01': '나 요새 친구들이랑 대화하는 게 어려워.', 'SS01': '친구들과 대화하는 것이 어려운 이유가 있을까요?', 'HS02': '친구가 날 안좋아 할지 걱정되고 내가 말실수는 하지 않았을까 두려워.', 'SS02': '친구와 대화하기 힘들어 많이 괴로우시겠어요.', 'HS03': '너무 속상하고 불편해.', 'SS03': '친구와 다시 편하게 대화하길 바라고 계시는군요.'}, {'HS01': '결혼 후 맞벌이를 하고 있는데 은퇴를 생각하면 어렵네.', 'SS01': '은퇴를 생각하며 걱정이 되시는군요. 은퇴를 위해 준비하신 게 있으신가요?', 'HS02': '노후대비를 생각해서 연금 저축을 들어놓은 게 있어.', 'SS02': '잘 준비하셨네요. 마음은 편안하신가요?', 'HS03': '

In [14]:
print(type(train_data_content[0]))

<class 'list'>


In [15]:
print(type(train_data_content[0][0]))

<class 'str'>


In [16]:
# train_data_content list -> str 로 이어붙이기
train_data_content2 = []

for idx in range(len(train_data_content)):
    train_data_content2.append('')
    
    for elem in train_data_content[idx]:
        train_data_content2[idx] += elem
    

In [17]:
print(train_data_content2[:5], type(train_data_content2[0]), type(train_data_content2[0][0])) # okay !

['기분이 좋지 않아기분이 좋지 않은 이유를 알 수 있을까요?요새 건강검진을 받았는데 건강 상태가 좋지 않아.많이 속상하시겠어요. 어디가 안 좋은가요?이곳 저곳 관리해야 할 곳이 많아.안좋은 곳이 많아 걱정이 많으시겠어요.', '진로가 너무 걱정스러워. 내가 진짜 뭘 해야 할지 모르겠어.미래가 두려우시군요. 하고 싶은 일이 있으세요?하고 싶은 일은 있는데 지금 시작하기에 너무 늦은 것 같아서 선뜻 도전을 못 하겠어.도전하는 것에 두려움을 느끼고 계시네요.', '아이가 친구랑 다퉈서 걱정이야.걱정이 되시겠군요. 아이는 어떤 상태인가요?아이도 걱정이 되는지 밥도 잘 안먹더라고.많이 걱정되시겠어요. 어떻게 하면 아이를 도와줄 수 있을까요?저녁먹고 대화를 해봐야지.대화가 잘 되었으면 좋겠어요.', '나 요새 친구들이랑 대화하는 게 어려워.친구들과 대화하는 것이 어려운 이유가 있을까요?친구가 날 안좋아 할지 걱정되고 내가 말실수는 하지 않았을까 두려워.친구와 대화하기 힘들어 많이 괴로우시겠어요.너무 속상하고 불편해.친구와 다시 편하게 대화하길 바라고 계시는군요.', '결혼 후 맞벌이를 하고 있는데 은퇴를 생각하면 어렵네.은퇴를 생각하며 걱정이 되시는군요. 은퇴를 위해 준비하신 게 있으신가요?노후대비를 생각해서 연금 저축을 들어놓은 게 있어.잘 준비하셨네요. 마음은 편안하신가요?이걸로 충분할지 잘 모르겠어. 불안한 마음이 자꾸 들어.미래에 대해 불안함으로 마음이 힘드시군요.'] <class 'str'> <class 'str'>


In [18]:
len(train_data_content2) # 총 15832 개의 대화

15832

In [20]:
# 대화 없는 발화 체크 (sanity check) -> okay !
count1 = 0

for elem in train_data_content2:
    if elem == '':
        count1 += 1
    
print(count1) # okay

0


In [21]:
print(train_data_content2[6000:6050])

['나는 아직도 더 일할 수 있는데 벌써 은퇴라니 정말 슬퍼.아직 일할 수 있는데 은퇴하게 되어서 속상하시군요.응. 몸도 건강하고 이곳에서 젊은 시절부터 얼마나 성실하게 일했는데 이제 나가라고 하네. 이제 어떻게 해야 할지 모르겠어.젊은 시절부터 일했는데 나가라고 해서 슬프셨군요.맞아. 하지만 어떡하겠어. 회사가 어려워져서 어쩔 수 없다고 하니. 좀 쉬다가 다른 일자리가 있는지 알아봐야지.쉬다가 다른 일자리를 알아보고자 하시는군요. 생각해보 신 일자리가 있으신가요?', '나 코로나인 줄 알았는데 그냥 독감이었어. 다행이고 기뻐.코로나가 아닌 독감이어서 기쁘셨군요.코로나였으면 어쩔 뻔했어. 실은 저번에 모임에서 잠깐 마스크를 벗었거든.모임에서 마스크를 벗어서 걱정하셨군요. 이제 감기에서 나으려면 어떻게 해야 할까요?외출하면 절대로 마스크를 벗지 않을 거야.마스크를 벗지 않으며 건강을 유지하셨으면 좋겠어요.', '젊을 때는 나이들어 부유할 줄 알았는데 현실은 정반대로 돈 걱정하는 모습이 한심하게 느껴져.돈 때문에 걱정하는 모습을 스스로 한심하게 느끼시는군요.노후 준비를 했어야 하는데 이제 와서 어떻게 해야 할지 막막해.그런 부분에 대해서 조언을 구할 상대가 있을까요?투자 전문가를 만나는 건 너무 벅찰 것 같아서 친구들에게 한 번 물어봐야겠어.친구와의 대화가 도움이 되면 좋겠군요.', '내가 연애에 너무 방어적인 사람이래.무슨 일이죠? 설명해 줄 수 있나요?그냥 내 행동이나 말이 상대방이 느끼기에 방어적이래. 난 그냥 상처받고 싶지 않아서 한 행동과 말인데.상처받지 않기 위해 한 행동과 말이 상대에겐 방어적으로 느껴졌군요.맞아. 내가 잘못된 거라는 생각은 안 하는데 세상에는 원래 다양한 입장과 사람이 있는 거잖아.방어적이란 건 잘못이 아니라고 생각하시는데 주변 반응 때문에 신경쓰이셨던 것처럼 느껴지네요.', '이번 생은 망했어. 내 생에 취업이란 없나 봐.취업 때문에 걱정이 많으신가요?아주 절망적인 상황이야. 난 이렇게 늙어 죽을 거야.취업 문제로 마음이 많이 안 

- dev 데이터 불러오기

In [22]:
# json 딕셔너리 파일에서 content 데이터만 추출

valid_data = []
valid_data_content = []
valid_y = []

# dev data - 대화내용 'content' & emotion 'type'불러오기

with open(valid_path) as json_file:
    json_data2 = json.load(json_file)

    for j in json_data2: # i : dict
        temp = (j.get('talk')).get('content')
        
        if '' not in temp:
            valid_data.append(temp) # "content" 만
            
            valid_data_content.append(list(temp.values()))
            
            #for v1 in temp.values():
            #    train_data_content += v1
            
            valid_y.append(((j.get('profile')).get('emotion')).get('type'))
        
        
    print(valid_data[:5], type(valid_data[0]))
    print()
    print(valid_y[:5], type(train_y[0]))
    print()
    print(valid_data_content[:5], type(valid_data_content[0]))

[{'HS01': '노후준비 하기가 정말 어렵네.', 'SS01': '제일 걱정이 뭐세요?', 'HS02': '돈이 제일 문제야.', 'SS02': '지금 노후준비 하고 계신건가요?', 'HS03': '적금이나 저축을 하긴 했는데 얼마 안되니까 문제지.', 'SS03': '저축을 해도 금액적으로 액수가 적은것 같다고 생각하시는군요.'}, {'HS01': '나이가 드니까 건강이 걱정스러워. 친한 친구가 암에 걸렸거든.', 'SS01': '친구가 암에 걸려서 건강에 대한 걱정이 많아지셨군요. 혹시 지금 아픈 곳이 있으신가요?', 'HS02': '아니. 아픈 곳은 없는데 건강 걱정이 늘었지.', 'SS02': '혹시 이 문제로 상담할 만한 전문가가 주변에 계실까요?', 'HS03': '', 'SS03': ''}, {'HS01': '남자친구랑 십 년 가까이 만났는데 결혼적령기가 되니 결혼해도 될지 고민되네.', 'SS01': '결혼을 생각할 때 걱정되는 부분이 있는 건가요?', 'HS02': '좋은 부분도 많지만 술 마시는 날은 연락이 안되는 문제가 있어서 걱정이야.', 'SS02': '술 마시면 불안하게 걱정을 시키군요.', 'HS03': '이런 사람이랑 결혼을 해도 될까?', 'SS03': '남자친구와 걱정되는 부분에 대해 고민을 나눠보세요.'}, {'HS01': '걱정이 하나 있어.', 'SS01': '어떤 걱정인지 여쭤봐도 될까요?', 'HS02': '다들 새로운 친구를 잘 사귀는데 나만 아무도 만나지 못하는 것 같아.', 'SS02': '새로운 인간 관계를 맺지 못해 걱정이 되시나봐요.', 'HS03': '응. 이러다 나만 혼자가 될 것 같아 불안해.', 'SS03': '불안감을 많이 느끼고 계시군요.'}, {'HS01': '요즘 젊은 직원들은 도대체 이해할 수 없어', 'SS01': '직원들과 무슨일이 있으셨나봐요.', 'HS02': '왜이렇게 다들 느긋한지 일에 대해 제대로 하는 것도 없는것 같고 나만 안달난 기분이야.', 'SS02': '혼자만 마음이 

In [23]:
print(type(valid_data_content[0]))

<class 'list'>


In [24]:
print(type(valid_data_content[0][0]))

<class 'str'>


In [25]:
# dev_data_content list -> str 로 이어붙이기
valid_data_content2 = []

for idx in range(len(valid_data_content)):
    valid_data_content2.append('')
    
    for elem in valid_data_content[idx]:
        valid_data_content2[idx] += elem
    

In [26]:
print(valid_data_content2[:5], type(valid_data_content2[0]), type(valid_data_content2[0][0])) # okay !

['노후준비 하기가 정말 어렵네.제일 걱정이 뭐세요?돈이 제일 문제야.지금 노후준비 하고 계신건가요?적금이나 저축을 하긴 했는데 얼마 안되니까 문제지.저축을 해도 금액적으로 액수가 적은것 같다고 생각하시는군요.', '나이가 드니까 건강이 걱정스러워. 친한 친구가 암에 걸렸거든.친구가 암에 걸려서 건강에 대한 걱정이 많아지셨군요. 혹시 지금 아픈 곳이 있으신가요?아니. 아픈 곳은 없는데 건강 걱정이 늘었지.혹시 이 문제로 상담할 만한 전문가가 주변에 계실까요?', '남자친구랑 십 년 가까이 만났는데 결혼적령기가 되니 결혼해도 될지 고민되네.결혼을 생각할 때 걱정되는 부분이 있는 건가요?좋은 부분도 많지만 술 마시는 날은 연락이 안되는 문제가 있어서 걱정이야.술 마시면 불안하게 걱정을 시키군요.이런 사람이랑 결혼을 해도 될까?남자친구와 걱정되는 부분에 대해 고민을 나눠보세요.', '걱정이 하나 있어.어떤 걱정인지 여쭤봐도 될까요?다들 새로운 친구를 잘 사귀는데 나만 아무도 만나지 못하는 것 같아.새로운 인간 관계를 맺지 못해 걱정이 되시나봐요.응. 이러다 나만 혼자가 될 것 같아 불안해.불안감을 많이 느끼고 계시군요.', '요즘 젊은 직원들은 도대체 이해할 수 없어직원들과 무슨일이 있으셨나봐요.왜이렇게 다들 느긋한지 일에 대해 제대로 하는 것도 없는것 같고 나만 안달난 기분이야.혼자만 마음이 급하신것 같아서 기분이 언짢아 보이시네요.'] <class 'str'> <class 'str'>


In [27]:
valid_y[:100]

['E37',
 'E37',
 'E37',
 'E37',
 'E37',
 'E24',
 'E31',
 'E13',
 'E24',
 'E36',
 'E19',
 'E12',
 'E31',
 'E21',
 'E11',
 'E12',
 'E43',
 'E49',
 'E43',
 'E44',
 'E39',
 'E17',
 'E37',
 'E32',
 'E44',
 'E12',
 'E53',
 'E21',
 'E12',
 'E44',
 'E48',
 'E47',
 'E13',
 'E40',
 'E61',
 'E50',
 'E61',
 'E61',
 'E61',
 'E44',
 'E61',
 'E15',
 'E26',
 'E65',
 'E59',
 'E64',
 'E43',
 'E61',
 'E67',
 'E32',
 'E28',
 'E20',
 'E47',
 'E40',
 'E21',
 'E32',
 'E20',
 'E48',
 'E64',
 'E21',
 'E57',
 'E55',
 'E17',
 'E51',
 'E34',
 'E64',
 'E39',
 'E51',
 'E48',
 'E13',
 'E18',
 'E58',
 'E35',
 'E64',
 'E34',
 'E55',
 'E23',
 'E24',
 'E49',
 'E48',
 'E63',
 'E42',
 'E53',
 'E61',
 'E50',
 'E39',
 'E26',
 'E68',
 'E68',
 'E23',
 'E12',
 'E59',
 'E26',
 'E36',
 'E27',
 'E19',
 'E65',
 'E17',
 'E34',
 'E22']

In [28]:
len(valid_data_content2) # 총 1717 번의 주고 받은 대화

1717

In [29]:
# 대화 없는 발화 체크 (sanity check) -> okay !
count = 0

for n in valid_data_content2:
    if n == '':
        count += 1
    
print(count) # okay !

0


In [30]:
# content 길이 간단한 EDA (train 데이터)

train_data_content_len = []
for a in train_data_content2:
    train_data_content_len.append(len(a))

train_data_content_len[:10]

[122, 128, 128, 148, 173, 170, 161, 209, 108, 192]

In [31]:
train_data_content_len = np.array(train_data_content_len)

print(train_data_content_len.mean(),
train_data_content_len.min(),
train_data_content_len.max()) # 최대 495 음절

167.71532339565437 37 495


In [32]:
# content 길이 간단한 EDA (valid 데이터)

valid_data_content_len = []
for a in valid_data_content2:
    valid_data_content_len.append(len(a))

valid_data_content_len[:10]

[115, 137, 175, 134, 119, 93, 129, 144, 151, 156]

In [33]:
valid_data_content_len = np.array(valid_data_content_len)

print(valid_data_content_len.mean(),
valid_data_content_len.min(),
valid_data_content_len.max()) # 최대 421 음절

167.10017472335468 46 421


In [34]:
print(len(train_y), len(valid_y))

15832 1717


In [35]:
type(train_y) # 원래 => list

list

In [36]:
type(train_y[0])

str

# 감정분류표에 맞게 class label 값 숫자로 치환 (0~59)

In [47]:
# 소 분류
emotion_code = {f'E{i+10}': i for i in range(60)}

In [48]:
# train
train_labels = []
for elem1 in train_y:
    for k, v in emotion_code.items():
        if elem1 == k:
            train_labels.append(v)
            #print(elem1)

In [49]:
# valid
validation_labels = []
for elem2 in valid_y:
    for k, v in emotion_code.items():
        if elem2 == k:
            validation_labels.append(v)

In [50]:
print(train_labels[:10])
print(validation_labels[:10])

[27, 27, 27, 27, 27, 27, 27, 27, 27, 27]
[27, 27, 27, 27, 27, 14, 21, 3, 14, 26]


In [51]:
print(len(train_labels), len(validation_labels)) # okay !

15832 1717


In [52]:
# 대 분류
import math

large_emotion_code = {f'E{i+10}': math.floor(i//10) for i in range(60)}
print(large_emotion_code)

{'E10': 0, 'E11': 0, 'E12': 0, 'E13': 0, 'E14': 0, 'E15': 0, 'E16': 0, 'E17': 0, 'E18': 0, 'E19': 0, 'E20': 1, 'E21': 1, 'E22': 1, 'E23': 1, 'E24': 1, 'E25': 1, 'E26': 1, 'E27': 1, 'E28': 1, 'E29': 1, 'E30': 2, 'E31': 2, 'E32': 2, 'E33': 2, 'E34': 2, 'E35': 2, 'E36': 2, 'E37': 2, 'E38': 2, 'E39': 2, 'E40': 3, 'E41': 3, 'E42': 3, 'E43': 3, 'E44': 3, 'E45': 3, 'E46': 3, 'E47': 3, 'E48': 3, 'E49': 3, 'E50': 4, 'E51': 4, 'E52': 4, 'E53': 4, 'E54': 4, 'E55': 4, 'E56': 4, 'E57': 4, 'E58': 4, 'E59': 4, 'E60': 5, 'E61': 5, 'E62': 5, 'E63': 5, 'E64': 5, 'E65': 5, 'E66': 5, 'E67': 5, 'E68': 5, 'E69': 5}


In [53]:
# train
train_labels_large = []
for elem1 in train_y:
    for k, v in large_emotion_code.items():
        if elem1 == k:
            train_labels_large.append(v)
            #print(elem1)

In [54]:
# valid
validation_labels_large = []
for elem2 in valid_y:
    for k, v in large_emotion_code.items():
        if elem2 == k:
            validation_labels_large.append(v)

In [55]:
print(train_labels_large[:10])
print(validation_labels_large[:10])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 1, 2, 0, 1, 2]


In [56]:
print(len(train_labels_large), len(validation_labels_large)) # okay !

15832 1717


# y_data categorical 인코딩

In [57]:
## 소 분류

In [58]:
train_labels = to_categorical(train_labels)
train_labels.shape

(15832, 60)

In [59]:
validation_labels = to_categorical(validation_labels)
validation_labels.shape

(1717, 60)

In [60]:
train_labels[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0

In [61]:
validation_labels[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0

In [62]:
## 대 분류

In [63]:
train_labels_large = to_categorical(train_labels_large)
train_labels_large.shape

(15832, 6)

In [64]:
validation_labels_large = to_categorical(validation_labels_large)
validation_labels_large.shape

(1717, 6)

In [65]:
train_labels_large[:10]

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [66]:
validation_labels_large[:10]

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

# 데이터 전처리

# Tokenizing

- Albert Pretrain 시키기 - kobert(monologg) 사용

In [70]:
!pip install transformers

     |████████████████████████████████| 1.5MB 10.8MB/s 
     |████████████████████████████████| 890kB 32.1MB/s 
     |████████████████████████████████| 2.9MB 39.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7cfc4e79d9bfb8efae36c165915c70b90a60ae5a247d6b046bb9be3cef123af0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [71]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

In [72]:
!pip install kobert_transformers

In [74]:
import torch
from kobert_transformers import get_kobert_model, get_distilkobert_model
model = get_distilkobert_model() # 변화 - distil 코버트 사용
model.eval()
#input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
#attention_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
#token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
#sequence_output, pooled_output = model(input_ids, attention_mask, token_type_ids)
#sequence_output[0]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in

In [75]:
print(model)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in

In [ ]:
# 1) transfer 그대로 평가???

In [ ]:
# 파이토치 임포트


In [76]:
# 2) 단어 사전만 활용해서 새로 토크나이징+모델링 ??

In [80]:
from kobert_transformers import get_tokenizer
tokenizer = get_tokenizer()
tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']

In [81]:
# 토큰화 + 숫자 치환(Word 2 Index)

# train
train_sequences =[]

for a in range(len(train_data_content2)):
    train_sequences.append(tokenizer.tokenize(train_data_content2[a]))
    train_sequences[a] = tokenizer.convert_tokens_to_ids(train_sequences[a])


# valid
valid_sequences =[]

for b in range(len(valid_data_content2)):
    valid_sequences.append(tokenizer.tokenize(valid_data_content2[b]))
    valid_sequences[b] = tokenizer.convert_tokens_to_ids(valid_sequences[b])

In [82]:
train_sequences[:5]

[[1282,
  7096,
  4210,
  3155,
  5561,
  6416,
  7096,
  4210,
  3162,
  3735,
  3166,
  2872,
  3881,
  6999,
  258,
  6999,
  6536,
  882,
  5392,
  7344,
  7088,
  2229,
  5761,
  882,
  2680,
  5330,
  4210,
  3155,
  54,
  6158,
  7096,
  2856,
  6527,
  7782,
  6705,
  5405,
  6857,
  54,
  3223,
  5330,
  3135,
  4209,
  5330,
  6999,
  258,
  7096,
  5451,
  3990,
  5451,
  1088,
  7852,
  4977,
  1022,
  1952,
  54,
  6812,
  7272,
  1022,
  1952,
  880,
  7096,
  517,
  6158,
  7074,
  6705,
  5405,
  6857,
  54],
 [4360,
  6079,
  5330,
  1458,
  880,
  6668,
  54,
  1435,
  4368,
  517,
  6249,
  5010,
  4977,
  7318,
  2049,
  5405,
  6855,
  54,
  6258,
  5330,
  1773,
  6060,
  7005,
  6705,
  5512,
  6999,
  54,
  4926,
  3075,
  3818,
  3854,
  7074,
  6586,
  258,
  7788,
  3075,
  3803,
  7086,
  3861,
  4299,
  2986,
  7789,
  6896,
  1458,
  517,
  5767,
  7086,
  905,
  832,
  6553,
  2734,
  5997,
  1721,
  7088,
  2086,
  4924,
  5405,
  6855,
  54,
  5859,
  7

In [83]:
valid_sequences[:5]

[[1493,
  7289,
  4927,
  5330,
  4102,
  3231,
  5702,
  54,
  7234,
  7126,
  880,
  7096,
  2145,
  6586,
  258,
  5866,
  7096,
  4128,
  7126,
  2125,
  6844,
  54,
  7323,
  1493,
  7289,
  4926,
  980,
  6733,
  5384,
  5330,
  6999,
  258,
  7202,
  5550,
  7098,
  517,
  7200,
  7088,
  4924,
  5584,
  5018,
  3252,
  3135,
  5886,
  5771,
  2125,
  7318,
  54,
  7200,
  7088,
  5002,
  1239,
  7203,
  517,
  6835,
  6630,
  4005,
  5398,
  831,
  5439,
  2705,
  7782,
  6705,
  5760,
  5512,
  6999,
  54],
 [1397,
  5330,
  1788,
  5771,
  882,
  7096,
  880,
  6668,
  54,
  4626,
  7828,
  4627,
  5330,
  3177,
  6896,
  889,
  6072,
  5377,
  5928,
  54,
  7489,
  5495,
  5330,
  3177,
  6896,
  889,
  6060,
  6553,
  882,
  6896,
  1682,
  880,
  7096,
  1952,
  7318,
  6604,
  5512,
  6999,
  54,
  517,
  7930,
  6705,
  4299,
  3093,
  7760,
  1022,
  3854,
  7074,
  6733,
  5330,
  6999,
  258,
  6797,
  5770,
  54,
  3093,
  7760,
  1021,
  7086,
  3272,
  5850,
  882,

In [84]:
from tensorflow.keras.preprocessing.text import Tokenizer # 전처리 관련
from tensorflow.keras.preprocessing.sequence import pad_sequences # 전처리 관련

In [85]:
# 시퀀스의 길이를 맞춰주기

# 한 문장의 최대 단어 숫자
max_len = 500

# 잘라낼 문장의 위치
trunc_type='post'

# 채워줄 문장의 위치
padding_type='post' # 'post' -> 뒷 부분 , 'pre' -> 앞 부분

In [86]:
train_padded = pad_sequences(train_sequences, maxlen=max_len, truncating=trunc_type, padding=padding_type)
# train_sentences 말고 숫자로 변환된 train_sequences를 넣어야 ! # truncating 옵션 : max_length 길이를 넘는 단어를 잘라내주는 옵션 (여기서는 'post'(뒤) 를 잘라냄) 
# padding 옵션 : max_length 길이 보다 짧은 단어는 값을 채워주는 옵션
validation_padded = pad_sequences(valid_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [87]:
train_padded.shape # (train 분할 단위 , max_length)
#type(train_padded)

(15832, 500)

In [88]:
train_padded.shape # (train 분할 단위 , max_length)
#type(train_padded)

(15832, 500)

In [89]:
train_padded.shape # (train 분할 단위 , max_length)
#type(train_padded)

(15832, 500)

# 데이터 shuffle
-x_train과 y_train을 배열로 묶어주고 shuffle ->> 그 결과를 다시 x_train, y_train 으로 나눈다

In [90]:
## 소 분류

In [91]:
tmp = [[x,y] for x, y in zip(train_padded, train_labels)]
tmp[:10]

[[array([1282, 7096, 4210, 3155, 5561, 6416, 7096, 4210, 3162, 3735, 3166,
         2872, 3881, 6999,  258, 6999, 6536,  882, 5392, 7344, 7088, 2229,
         5761,  882, 2680, 5330, 4210, 3155,   54, 6158, 7096, 2856, 6527,
         7782, 6705, 5405, 6857,   54, 3223, 5330, 3135, 4209, 5330, 6999,
          258, 7096, 5451, 3990, 5451, 1088, 7852, 4977, 1022, 1952,   54,
         6812, 7272, 1022, 1952,  880, 7096,  517, 6158, 7074, 6705, 5405,
         6857,   54,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0

In [92]:
import random
random.shuffle(tmp)
tmp[:10]

[[array([1370, 5102, 3501, 2443, 6903, 3803, 7794, 1549, 7096, 6844,   54,
         6247, 1849, 1204, 1546, 6705, 5655, 6999,  258, 7094, 3224, 3818,
         1810, 6198, 1370, 5102, 4930, 1282, 7096, 6844,   54, 6855, 5975,
         5400, 4939, 3803, 2428, 4252, 2872, 3881, 6999,  258, 6854, 5568,
         4998, 2421, 5761, 1562, 5931, 3890, 5933, 4966, 1198, 5839, 5439,
           54, 6542, 5336, 7096, 1956, 1598, 6553, 2856, 6527, 7782, 6738,
          905,  832, 6999,   54,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0

In [93]:
train_padded = [n[0] for n in tmp]
train_labels = [n[1] for n in tmp]

In [94]:
# check
train_padded[:10]

[array([1370, 5102, 3501, 2443, 6903, 3803, 7794, 1549, 7096, 6844,   54,
        6247, 1849, 1204, 1546, 6705, 5655, 6999,  258, 7094, 3224, 3818,
        1810, 6198, 1370, 5102, 4930, 1282, 7096, 6844,   54, 6855, 5975,
        5400, 4939, 3803, 2428, 4252, 2872, 3881, 6999,  258, 6854, 5568,
        4998, 2421, 5761, 1562, 5931, 3890, 5933, 4966, 1198, 5839, 5439,
          54, 6542, 5336, 7096, 1956, 1598, 6553, 2856, 6527, 7782, 6738,
         905,  832, 6999,   54,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [95]:
type(train_padded)

list

In [96]:
train_labels[:10]

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [97]:
# check
print(train_labels.shape, validation_labels.shape)

AttributeError: ignored

In [ ]:
## 대 분류

In [ ]:
tmp2 = [[x,y] for x, y in zip(train_padded, train_labels_large)]
tmp2[:10]

In [98]:
import random
random.shuffle(tmp2)
tmp2[:10]

NameError: ignored

In [ ]:
train_padded = [n[0] for n in tmp2]
train_labels_large = [n[1] for n in tmp2]

In [99]:
# check
train_padded[:10]

[array([1370, 5102, 3501, 2443, 6903, 3803, 7794, 1549, 7096, 6844,   54,
        6247, 1849, 1204, 1546, 6705, 5655, 6999,  258, 7094, 3224, 3818,
        1810, 6198, 1370, 5102, 4930, 1282, 7096, 6844,   54, 6855, 5975,
        5400, 4939, 3803, 2428, 4252, 2872, 3881, 6999,  258, 6854, 5568,
        4998, 2421, 5761, 1562, 5931, 3890, 5933, 4966, 1198, 5839, 5439,
          54, 6542, 5336, 7096, 1956, 1598, 6553, 2856, 6527, 7782, 6738,
         905,  832, 6999,   54,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [100]:
type(train_padded)

list

In [101]:
train_labels_large[:10]

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

# 모델링

In [103]:
## 사전 조정 (소 분류)

train_input = np.array(train_padded)
train_labels = np.array(train_labels)

test_input = np.array(validation_padded)
test_labels = np.array(validation_labels)



print(train_input.shape, train_labels.shape, test_input.shape, test_labels.shape)

(15832, 500) (15832, 60) (1717, 500) (1717, 60)


In [104]:
##### 임포트
from keras import optimizers, losses
from keras.optimizers import Adam, SGD

In [105]:
vocab_size =8002
embedding_dim = 64

In [106]:
##### 모델 4 -캐글 TCN
# https://www.kaggle.com/abhiramanand/project-tcn


####
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    
    #tf.keras.layers.Conv1D(filters=64, kernel_size=20,strides=10,padding='valid'), #kernel_size=10

    tf.keras.layers.Conv1D(filters=1024, kernel_size=50,strides=1,padding='causal', activation='relu'), #kernel_size=100
    
    #tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    #tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.GlobalAveragePooling1D(),
    
    
    tf.keras.layers.Dense(60, activation='softmax')
])

In [107]:
#### 수정
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    
    #tf.keras.layers.Conv1D(filters=64, kernel_size=20,strides=10,padding='valid'), #kernel_size=10

    tf.keras.layers.Conv1D(filters=60, kernel_size=20,strides=10,padding='causal', activation='relu'), #kernel_size=100
    
    #tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    #tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.GlobalAveragePooling1D(),
    
    
    tf.keras.layers.Dense(60, activation='softmax')
])

In [108]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 64)           512128    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 60)            76860     
_________________________________________________________________
dense_2 (Dense)              (None, 50, 128)           7808      
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 60)                7740      
Total params: 604,536
Trainable params: 604,536
Non-trainable params: 0
_________________________________________________________________


In [109]:
# learing rate 옵션 (adam)

#1) SGD
#opt = optimizers.SGD(lr=0.003, momentum=0.0, decay=0.01, nesterov=False)

#2) Adam
opt = optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False) # learing rate 옵션

#keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

In [110]:
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss',
                             verbose=1)

In [111]:
# fit model
num_epochs = 10

history = model.fit(train_input, train_labels, 
                    validation_data = (test_input, test_labels),
                    callbacks = [checkpoint],
                    epochs=num_epochs)#, verbose=2) #validation_split=0.2)

Epoch 1/10
495/495 [==============================] - 20s 38ms/step - loss: 4.0740 - acc: 0.0205 - val_loss: 3.8380 - val_acc: 0.0635

Epoch 00001: val_loss improved from inf to 3.83799, saving model to my_checkpoint.ckpt
Epoch 2/10
495/495 [==============================] - 19s 38ms/step - loss: 3.5872 - acc: 0.1009 - val_loss: 3.4824 - val_acc: 0.1602

Epoch 00002: val_loss improved from 3.83799 to 3.48242, saving model to my_checkpoint.ckpt
Epoch 3/10
495/495 [==============================] - 20s 41ms/step - loss: 3.0659 - acc: 0.2188 - val_loss: 3.4247 - val_acc: 0.1771

Epoch 00003: val_loss improved from 3.48242 to 3.42466, saving model to my_checkpoint.ckpt
Epoch 4/10
495/495 [==============================] - 19s 39ms/step - loss: 2.7516 - acc: 0.2961 - val_loss: 3.4890 - val_acc: 0.1881

Epoch 00004: val_loss did not improve from 3.42466
Epoch 5/10
495/495 [==============================] - 19s 38ms/step - loss: 2.4547 - acc: 0.3672 - val_loss: 3.6197 - val_acc: 0.1864

Epoch

In [112]:
model.load_weights(checkpoint_path) # 가중치 로드

In [113]:
## 사전 조정 (대 분류)

train_input = np.array(train_padded)
train_labels_large = np.array(train_labels_large)

test_input = np.array(validation_padded)
test_labels_large = np.array(validation_labels_large)



print(train_input.shape, train_labels_large.shape, test_input.shape, test_labels_large.shape)

(15832, 500) (15832, 6) (1717, 500) (1717, 6)


In [114]:
## 대 분류 추가 학습

In [115]:
#### 수정 (대분류)
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    
    #tf.keras.layers.Conv1D(filters=64, kernel_size=20,strides=10,padding='valid'), #kernel_size=10

    tf.keras.layers.Conv1D(filters=60, kernel_size=20,strides=10,padding='causal', activation='relu'), #kernel_size=100
    
    #tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    #tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.GlobalAveragePooling1D(),
    
    
    tf.keras.layers.Dense(6, activation='softmax')
])

In [116]:
# learing rate 옵션 (adam)

#1) SGD
#opt = optimizers.SGD(lr=0.003, momentum=0.0, decay=0.01, nesterov=False)

#2) Adam
opt = optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False) # learing rate 옵션

#keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

In [117]:
#checkpoint_path = 'my_checkpoint.ckpt'
checkpoint2 = ModelCheckpoint(checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss',
                             verbose=1)

In [118]:
# fit model
num_epochs = 10

history = model2.fit(train_input, train_labels_large, 
                    validation_data = (test_input, test_labels_large),
                    callbacks = [checkpoint2],
                    epochs=num_epochs)#, verbose=2) #validation_split=0.2)

Epoch 1/10
495/495 [==============================] - 20s 38ms/step - loss: 1.7923 - acc: 0.1664 - val_loss: 1.7918 - val_acc: 0.1683

Epoch 00001: val_loss improved from inf to 1.79176, saving model to my_checkpoint.ckpt
Epoch 2/10
495/495 [==============================] - 19s 38ms/step - loss: 1.7877 - acc: 0.1862 - val_loss: 1.8062 - val_acc: 0.1776

Epoch 00002: val_loss did not improve from 1.79176
Epoch 3/10
495/495 [==============================] - 19s 38ms/step - loss: 1.4885 - acc: 0.4394 - val_loss: 2.0454 - val_acc: 0.1765

Epoch 00003: val_loss did not improve from 1.79176
Epoch 4/10
495/495 [==============================] - 19s 38ms/step - loss: 0.9139 - acc: 0.6847 - val_loss: 2.6342 - val_acc: 0.1701

Epoch 00004: val_loss did not improve from 1.79176
Epoch 5/10
495/495 [==============================] - 19s 38ms/step - loss: 0.5649 - acc: 0.8289 - val_loss: 3.1817 - val_acc: 0.1660

Epoch 00005: val_loss did not improve from 1.79176
Epoch 6/10
495/495 [==============

In [ ]:
model2.load_weights(checkpoint_path) # 가중치 로드

In [ ]:
#### test 비공개 데이터 예측

In [ ]:
pred = model.predict_proba(test_input)